In [23]:
import math
import numpy as np
import pandas as pd
from scipy.stats import linregress

nInst=100
currentPos = np.zeros(nInst)

# A basic theoretical price function
def getTheoreticalPrices(prcSoFar):
    # an array containing the latest prices of each stockie (I DONT KNOW HOW TRY AND EXCEPT WORK!)
    try:
        lastPrice = prcSoFar[:,-1]
        secondLastPrice = prcSoFar[:,-2]
    except:
        # Handle Day 1 when prcSoFar is a 1D array
        lastPrice = prcSoFar
        secondLastPrice = prcSoFar
    
    #RSIvalues = applyRSIModel(prcSoFar)
    #weightfactor = 1/3000 # 
    
    # The theoretical price is the last price with a basic mean reversion - 0.5 of the way to the 2nd last price
    theoreticalPrices = lastPrice + (secondLastPrice - lastPrice) * 0.8 #\
    #+ (50*np.ones(100) - rsi_values) * lastPrice * weightfactor
    return theoreticalPrices


# Applys the edge model to determine how much we should size up/down for each stockie
def applyEdgeModel(prcSoFar, theoPrices, currentPos): # theoPrices is a 100 length array with the edge for each

    # an array containing the latest prices of each stockie (I DONT KNOW HOW TRY AND EXCEPT WORK!)
    try:
        lastPrice = prcSoFar[:,-1]
    except:
        # Handle Day 1 when prcSoFar is a 1D array
        lastPrice = prcSoFar

    # Our edge is the theoretical price minus the current price expressed as a percentage (positive means buy)
    edgeArray = (theoPrices - lastPrice)/lastPrice
    newPosition = currentPos
    edgeRequired = 0.01 # if we have 1% of edge we will max out long and short based on direction
    for i in range(len(edgeArray)):
        edge = edgeArray[i]
        if (edge > edgeRequired).any():
            newPosition[i] = math.floor(10000/lastPrice[i]) # We buy as much as we allowed! Everything!
        if (edge < -edgeRequired).any():
            newPosition[i] = math.ceil(-10000/lastPrice[i]) # We sell as much as we allowed! Everything!
    return newPosition

# Function to change current position based on the results of the RSI Model
# Calculates the Relative Strength Index and uses the last RSI to determine the current stock position
def applyRSIModel(prcSoFar):
    
    # RSI Method
    prcSoFar = pd.DataFrame(prcSoFar).T
    RSI = prcSoFar.copy()
    for i in range(100):
        RSI[i] = computeRSI(prcSoFar[i], 14)
        RSI_copy = RSI.fillna(0).to_numpy()
    
    (row,col) = RSI[i].shape
    print(f'Row: {row}, Col: {col}')
    assert len(RSI[-1]) == 100, "Position is incorrect"
    return np.transpose(RSI_copy[-1])

# Dummy algorithm to demonstrate function format.
def getMyPosition(prcSoFar):
    global currentPos
    (nins,nt) = prcSoFar.shape
    #rpos = np.array([int(x) for x in 1000 * np.random.randn(nins)])
    #currentPos += rpos

    #currentPos = applyRSIModel(prcSoFar, currentPos)
    
    theoPrices = getTheoreticalPrices(prcSoFar)
    currentPos = applyEdgeModel(prcSoFar, theoPrices, currentPos)
    # The algorithm must return a vector of integers, indicating the position of each stock.
    # Position = number of shares, and can be positve or negative depending on long/short position.
    return currentPos

# Function to check that the position does not exceed 10k limit per stock
# Input: currentPos, a 1*100 nparray & latest price of each stock
# Output: True (Valid Position) or False (at least one stock exceeding 10k limit)
def isPositionValid(currentPos, prcSoFar):
    limit = 10000
    result = True

    try:
        lastPrice = prcSoFar[-1,:]
    except:
        # Handle Day 1 when prcSoFar is a 1D array
        lastPrice = prcSoFar

    assert lastPrice.shape == currentPos.shape, "The dimension of the current position does not match data."
    
    positionValue = abs(currentPos)*lastPrice

    stockNumber = 0
    for pos in positionValue:
        if pos > limit:
            result = False
            print(f"Stock {stockNumber}'s position exceeds limit")
        stockNumber += 1
        assert stockNumber < 100, "More stocks than expected."

    return result

# Calculate the momentum for one stock given the stocks historical data
# Code from https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/
# Formula of momentum is the annualised exponential regression slope multiplied
# by the R^2 coefficient of the regression calculation
def momentum(prices):
    returns = np.log(prices)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)

# Calculating the RSI index for each stock 
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff<0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi


In [24]:
# Algorithm testing file. 
# Quantitative judging will be determined from output of this program.
# Judging will use unseeen, future price data from the same universe.

nInst = 0
nt = 0

# Commission rate.
commRate = 0.0050

# Dollar position limit (maximum absolute dollar value of any individual stock position).
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./prices250.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolume0 = 0
    totDVolume1 = 0
    frac0 = 0.
    frac1 = 0.
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(1,200):
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] 
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.array([int(p) for p in np.clip(newPosOrig, -posLimits, posLimits)])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume0 = np.sum(dvolumes[:50])
        dvolume1 = np.sum(dvolumes[50:])
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        totDVolume0 += dvolume0
        totDVolume1 += dvolume1
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
            frac0 = totDVolume0 / totDVolume
            frac1 = totDVolume1 / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf frac0: %.4lf frac1: %.4lf" % (t,value, todayPL, totDVolume, ret, frac0, frac1))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = 16 * plmu / plstd
    return (plmu, ret, annSharpe, totDVolume)

# Output.
(meanpl, ret, sharpe, dvol) = calcPL(prcAll)
print ("=====")
print ("mean(PL): %.0lf" % meanpl)
print ("return: %.5lf" % ret)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)

Loaded 100 instruments for 250 days
Day 1 value: -4995.24 todayPL: $-4995.24 $-traded: 999048 return: -0.00500 frac0: 0.4999 frac1: 0.5001
Day 2 value: -4057.64 todayPL: $937.60 $-traded: 1536338 return: -0.00264 frac0: 0.3644 frac1: 0.6356
Day 3 value: 2790.60 todayPL: $6848.24 $-traded: 2125221 return: 0.00131 frac0: 0.2918 frac1: 0.7082
Day 4 value: 7390.04 todayPL: $4599.43 $-traded: 2671471 return: 0.00277 frac0: 0.2546 frac1: 0.7454
Day 5 value: 17782.86 todayPL: $10392.83 $-traded: 3273709 return: 0.00543 frac0: 0.2144 frac1: 0.7856
Day 6 value: 25554.18 todayPL: $7771.32 $-traded: 3917957 return: 0.00652 frac0: 0.1997 frac1: 0.8003
Day 7 value: 36580.17 todayPL: $11025.99 $-traded: 4601747 return: 0.00795 frac0: 0.1876 frac1: 0.8124
Day 8 value: 49382.92 todayPL: $12802.74 $-traded: 5379940 return: 0.00918 frac0: 0.1790 frac1: 0.8210
Day 9 value: 56638.83 todayPL: $7255.91 $-traded: 5925196 return: 0.00956 frac0: 0.1694 frac1: 0.8306
Day 10 value: 61214.10 todayPL: $4575.27 $-t

Day 160 value: 973835.57 todayPL: $2363.85 $-traded: 95731039 return: 0.01017 frac0: 0.0965 frac1: 0.9035
Day 161 value: 982074.01 todayPL: $8238.44 $-traded: 96399933 return: 0.01019 frac0: 0.0962 frac1: 0.9038
Day 162 value: 985322.77 todayPL: $3248.76 $-traded: 96926698 return: 0.01017 frac0: 0.0964 frac1: 0.9036
Day 163 value: 990863.12 todayPL: $5540.35 $-traded: 97423036 return: 0.01017 frac0: 0.0959 frac1: 0.9041
Day 164 value: 996557.82 todayPL: $5694.70 $-traded: 98012996 return: 0.01017 frac0: 0.0957 frac1: 0.9043
Day 165 value: 1001442.40 todayPL: $4884.59 $-traded: 98635067 return: 0.01015 frac0: 0.0955 frac1: 0.9045
Day 166 value: 1008850.17 todayPL: $7407.76 $-traded: 99200909 return: 0.01017 frac0: 0.0958 frac1: 0.9042
Day 167 value: 1012866.77 todayPL: $4016.60 $-traded: 99690127 return: 0.01016 frac0: 0.0958 frac1: 0.9042
Day 168 value: 1018261.84 todayPL: $5395.07 $-traded: 100231081 return: 0.01016 frac0: 0.0953 frac1: 0.9047
Day 169 value: 1024627.29 todayPL: $6365.

In [3]:
prcSoFar = prcAll

In [8]:
try:
        lastPrice = prcSoFar[:,-1]
        secondLastPrice = prcSoFar[:,-2]
except:
        # Handle Day 1 when prcSoFar is a 1D array
        lastPrice = prcSoFar
        secondLastPrice = prcSoFar
        
# The theoretical price is the last price with a basic mean reversion - 0.5 of the way to the 2nd last price
theoreticalPrices = lastPrice + (secondLastPrice - lastPrice) * 0.5
theoreticalPrices


array([19.2  ,  7.39 , 19.145,  0.455,  2.35 ,  8.34 , 16.54 , 21.195,
       20.21 ,  2.595, 33.805,  1.06 ,  7.795, 14.995, 27.365, 29.77 ,
       30.24 , 24.295,  5.51 , 42.495, 14.23 ,  1.8  ,  7.12 ,  9.895,
        6.06 , 48.71 , 14.93 , 16.705, 12.34 , 26.14 , 25.58 , 17.925,
       14.8  ,  6.515, 30.8  ,  8.11 , 16.63 , 11.86 , 23.47 , 23.705,
        8.34 , 69.68 , 17.93 , 82.965, 35.8  , 60.325, 66.345, 26.435,
       37.575, 41.935, 13.08 ,  9.01 , 17.52 , 12.505, 18.15 ,  9.025,
       15.66 , 16.54 , 22.675,  0.52 , 14.705, 21.545, 10.075, 16.135,
       18.525, 20.695, 15.065, 15.45 , 10.115, 26.195, 15.435, 30.01 ,
       27.465, 24.875,  8.995, 13.335, 18.31 , 19.13 , 21.47 ,  6.47 ,
       19.215, 28.74 , 17.575, 19.385,  3.815, 26.835, 18.82 , 20.605,
       29.04 , 11.395,  7.635, 19.135, 19.845, 20.04 , 27.22 , 16.   ,
       19.83 ,  3.355,  2.545, 19.01 ])

In [5]:
# an array containing the latest prices of each stockie (I DONT KNOW HOW TRY AND EXCEPT WORK!)
try:
    lastPrice = prcAll[:,-1]
except:
    # Handle Day 1 when prcSoFar is a 1D array
    lastPrice = prcAll

# Our edge is the theoretical price minus the current price expressed as a percentage (positive means buy)
edgeArray = (theoreticalPrices - lastPrice)/lastPrice
len(edgeArray)
newPosition = currentPos
edgeRequired = 0.01 # if we have 1% of edge we will max out long and short based on direction
for i in range(len(edgeArray)):

    if edgeArray[i] > edgeRequired:
        newPosition[i] = math.floor(10000/lastPrice[i]) # We buy as much as we allowed! Everything!
    if edgeArray[i] < -edgeRequired:
        newPosition[i] = math.ceil(-10000/lastPrice[i]) # We sell as much as we allowed! Everything!
        
        
newPosition

array([ -509., -1362.,  -557., 22222., -4149.,  1240.,   625.,  -465.,
        -601., -3846.,  -293., -9009., -1248.,   647.,  -333.,  -360.,
        -327.,   434., -1992.,  -253.,  -674., -5434., -1408.,   964.,
       -1745.,  -210.,  -731.,  -592.,   844.,  -389.,  -374.,  -556.,
         610., -1628.,  -341., -1204.,  -568.,  -894.,  -426.,  -437.,
       -1256.,  -142.,  -446.,   125.,  -252.,  -172.,   151.,  -367.,
        -273.,  -212.,  -754.,  1129.,   599.,  -771.,  -535.,  1124.,
        -646.,  -582.,   458., 20000.,   675.,  -464.,   997.,  -597.,
        -555.,  -464.,   673.,   653.,  1010.,  -377.,  -643.,   344.,
         376.,   407.,  1127.,   773.,  -538.,   526.,  -440., -1550.,
         531.,   344.,  -554.,  -505., -2610.,   371.,  -505.,   491.,
        -337.,  -867.,  1432.,   531.,  -497.,   503.,  -367.,  -619.,
         517., -2976.,  4000.,   528.])